This notebook creates a simple modflow model, following the tutorials https://flopy.readthedocs.io/en/stable/tutorials.html#modflow-6

In [ ]:
import flopy

In [ ]:
# set paths and names
folder_name = "mf6_model"
gwf_name = 'gwf'
gwe_name = 'gwe'
ws = f"./{folder_name}"

### Model parameters

# Spatial and Temporal Discretization
nlay, nrow, ncol = 2, 1, 1
shape3d = (nlay, nrow, ncol)
shape2d = (nrow, ncol)
delr, delc = 1., 1
area = delr * delc
aquifer_thickness = 10.
dz = aquifer_thickness / nlay
elevations = [26.0, 26.0 - dz, 26.0 - 2*dz]
nper, pertime, nstp, tsmult = 10, 1, 48, 1

# Hydraulic Properties
k, ss, sy = 0.1, 1.5e-5, 0.05

# Initial Conditions
h0 = elevations[0] - 2.5

In [ ]:
### Build a one-dimensional model

# Simulation Object
sim = flopy.mf6.MFSimulation(sim_name=folder_name, sim_ws=ws, verbosity_level=1, memory_print_option="all", exe_name = exe_name)
tdis = flopy.mf6.ModflowTdis(sim, filename=f"{gwf_name}.tdis", time_units="days", nper=nper, perioddata=((pertime, nstp, tsmult),)*nper,)

# Groundwater Flow Model
gwf = flopy.mf6.ModflowGwf(sim, modelname=gwf_name, newtonoptions="NEWTON UNDER_RELAXATION", save_flows=True)
ims_gwf = flopy.mf6.ModflowIms(sim,  print_option="summary", complexity="MODERATE", filename=f"{gwf_name}.ims",
                               outer_maximum= 500, under_relaxation="dbd",
                               linear_acceleration="bicgstab", rcloserecord=[1e-6, "strict"],inner_dvclose=1e-8,outer_dvclose=1e-9,)
sim.register_ims_package(ims_gwf, [gwf.name]) 

is_gwf = flopy.mf6.ModflowGwfdis(gwf, length_units="meters", nlay=nlay, nrow=nrow, ncol=ncol, 
                        delr=delr, delc=delc, top=elevations[0], botm=elevations[1:])
npf = flopy.mf6.ModflowGwfnpf(gwf, icelltype=1, k=k)
sto = flopy.mf6.ModflowGwfsto(gwf, iconvert=1, ss=ss, sy=sy)
ic = flopy.mf6.ModflowGwfic(gwf, strt=h0)
rch = flopy.mf6.ModflowGwfrch(gwf, pname="rch_0", maxbound=10, save_flows=True, stress_period_data = [((0, 0, 0), 0.0001, 0)], auxiliary=["TEMPERATURE"])
oc_gwf = flopy.mf6.ModflowGwfoc(gwf, budget_filerecord = '{}.cbc'.format(gwf_name), 
                            budgetcsv_filerecord = '{}.cbc.csv'.format(gwf_name),
                            head_filerecord = '{}.hds'.format(gwf_name),
                            saverecord = [('HEAD', 'ALL'),('BUDGET', 'LAST')],
                            printrecord = [('HEAD', 'ALL'),('BUDGET', 'LAST')])
obs_lst = []
for k in range(nlay):
    obs_lst.append(["H{:02d}".format(k+1), "HEAD", (k, 0, 0)])
obs_gwf = flopy.mf6.ModflowUtlobs(gwf, print_input=False, continuous={"gwhead.csv": obs_lst})

# Groundwater Heat Model
gwe = flopy.mf6.ModflowGwe(sim, modelname=gwe_name, save_flows=True)
ims_gwe = flopy.mf6.ModflowIms(sim,  print_option="summary", filename=f"{gwe_name}.ims", complexity="SIMPLE",
                           linear_acceleration="bicgstab", rcloserecord=[1e-6, "strict"],inner_dvclose=1e-8,outer_dvclose=1e-9,);
sim.register_ims_package(ims_gwe, [gwe.name])

In [ ]:
# Instantiating MODFLOW 6 transport discretization package

strt_temp = 10.0  # Initial temperature ($^{\circ}C$)
scheme = "Upstream"  # Advection scheme ($-$)
alh = 0.0  # No mechanical dispersion ($m^2/day$)
ath1 = 0.0  # No transverse dispersivity ($m^2/day$)
dispersivity = 0.0  # Longitudinal mechanical dispersion term ($m$)
porosity = 0.2  # Porosity ($-$)
rhos = 1500.0  # Density of dry solid aquifer material ($\frac{kg}{m^3}$)
cps = 760.0  # Heat capacity of dry solid aquifer material ($\frac{J}{kg \cdot ^{\circ} C}$)
rhow = 1000.0  # Density of water ($\frac{kg}{m^3}$)
cpw = 4183.0  # Heat capacity of water ($\frac{J}{kg \cdot ^{\circ} C}$)
lhv = 2500.0  # Latent heat of vaporization
ktw = 0.5918  # Thermal conductivity of water ($\frac{W}{m \cdot ^{\circ} C}$)
kts = 0.27  # Thermal conductivity of solid aquifer material ($\frac{W}{m \cdot ^{\circ} C}$)

is_gwe = flopy.mf6.ModflowGwedis(gwe, length_units="meters", nlay=nlay, nrow=nrow, ncol=ncol, 
                        delr=delr, delc=delc, top=elevations[0], botm=elevations[1:])

# Instantiating MODFLOW 6 transport initial concentrations
ic_gwe =  flopy.mf6.ModflowGweic(gwe, strt=strt_temp, pname="IC", filename=f"{gwe_name}.ic",)

# Instantiating MODFLOW 6 transport advection package
adv = flopy.mf6.ModflowGweadv(gwe, scheme=scheme, pname="ADV", filename="{}.adv".format(gwe_name))

# Instantiating MODFLOW 6 transport dispersion package
cnd = flopy.mf6.ModflowGwecnd(gwe, xt3d_off=False, alh=alh, ath1=ath1, ktw=ktw * 86400, 
                              kts=kts * 86400, pname="CND",filename=f"{gwe_name}.cnd",)

# Instantiating MODFLOW 6 transport mass storage package
est = flopy.mf6.ModflowGweest(gwe, save_flows=True, porosity=porosity, cps=cps, rhos=rhos,
                              packagedata=[cpw, rhow, lhv], pname="EST", filename=f"{gwe_name}.est",)

# Instantiating MODFLOW 6 source/sink mixing package
sourcerecarray = [ ("rch_0", "AUX", "TEMPERATURE") ]
ssm = flopy.mf6.ModflowGwessm(gwe, sources = sourcerecarray, filename = "{}.ssm".format(gwe_name))

oc_gwe = flopy.mf6.ModflowGweoc(gwe, budget_filerecord = '{}.cbc'.format(gwe_name), 
                            budgetcsv_filerecord = '{}.cbc.csv'.format(gwe_name),
                            temperature_filerecord="{}.ucn".format(gwe_name),
                            saverecord = [('TEMPERATURE', 'LAST'),('BUDGET', 'LAST')],
                            printrecord = [('TEMPERATURE', 'LAST'),('BUDGET', 'LAST')])
gwfgwe = flopy.mf6.ModflowGwfgwe(sim, exgtype='GWF6-GWE6', exgmnamea = gwf_name, exgmnameb = gwe_name, filename = 'gwf_gwe.gwfgwe')

In [ ]:
# Write the Model Files
sim.write_simulation()